In [1]:
#准备数据
import numpy as np
print('preparing data...')
from sklearn import datasets
import numpy as np
iris=datasets.load_iris()
iris.keys()
X=iris['data']
y=(iris['target']).astype(np.int)
print('done!')
X.shape

preparing data...
done!


(150, 4)

In [3]:
 def fit(X,y,theta=None):
    eta=0.1
    sort_number=3   #sortnumber代表类别个数 以及类别标签
    if theta==None:
        theta=np.random.rand(X.shape[1],sort_number)
        old_theta=np.zeros((X.shape[1],sort_number))
    for i in range(100):
        old_theta=theta
        for k in range(sort_number):
            yk=y==k
            yk=yk.astype(int)
            s=(old_theta.T).dot(X.T)
            sk=s[k,:]
            s_sum=np.exp(s[0,:])
            for j in range(sort_number-1):
                s_sum=s_sum+np.exp(s[j+1,:])
            pk=np.exp(sk)/s_sum
            delta=1/len(X)*((pk-yk)).dot(X)
            theta[:,k]=old_theta[:,k]-eta*delta
    return theta
def predict(x,theta):
    pro=np.exp(theta.T.dot(x.T))/sum(np.exp(theta.T.dot(x.T)))
    return np.argmax(pro,axis=0)
def early_stopping_predict(X,y,x):
    from sklearn.model_selection import train_test_split
    x_train,x_val,y_train,y_val=train_test_split(X,y,test_size=0.6)
    best_theta=None
    best_epoch=None
    min_val_error=float('inf')
    for epoch in range(80):
        theta=fit(x_train,y_train)
        train_mean_error=(((predict(x_train,theta)-y_train)**2).sum()/len(x_train))**0.5
        val_mean_error=(((predict(x_val,theta)-y_val)**2).sum()/len(x_val))**0.5
        if val_mean_error<min_val_error:
            best_theta=theta
            best_epoch=epoch
            min_val_error=val_mean_error
            best_train_error=train_mean_error
    print('train_error:',best_train_error,'val_error:',min_val_error,'best_epoch:',best_epoch)
    return predict(x,best_theta)
pre_y=early_stopping_predict(X,y,X)
pre_y,sum(pre_y==y)/len(y)

train_error: 0.18257418583505536 val_error: 0.14907119849998599 best_epoch: 8


(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
 0.9733333333333334)

In [4]:
from collections import Counter
import numpy as np
#决策树

class s_data(object):
    def __init__(self, x, y):
        self.x = x
        self.y = y

class node(object):
    def __init__(self, data, index_list):
        self.data = data
        self.clas = Counter(data.y).most_common()[0][0]
        self.index_list = index_list

class condition_node(object):
    def __init__(self, f_index, index_list, data,alpha):
        self.alpha=alpha
        self.data = data
        self.f_index = f_index
        self.index_list = index_list
        self.clas = Counter(data.y).most_common()[0][0]

    def create_nodes(self, data):
        self.branches = dict()
        for f in set(data.x.T[self.f_index]):
            self.branches[f] = create_tree(s_data(
                data.x[data.x.T[self.f_index] == f], data.y[data.x.T[self.f_index] == f]), self.index_list,alpha=self.alpha)

    def judge(self, x):
        return self.data_list[x[self.f_index]]

def claculate_H_D(data):
    H_D = 0
    for yi in set(data.y):
        Ck_D = (data.y == yi).sum() / len(data.y)
        H_D += -Ck_D * np.log(Ck_D)
    return H_D

def claculate_H_D_A(data, A):
    H_D_A = 0
    for a in set(data.x.T[A]):
        H_D_A += (data.x.T[A] == a).sum() / len(data.y) * claculate_H_D(
            s_data(data.x[data.x.T[A] == a], data.y[data.x.T[A] == a]))
    return H_D_A


def claculate_max_g(data, index_list,alpha):
    max_index = index_list[0]
    max_g = 0
    H_D = claculate_H_D(data)
    for index in index_list:
        g_D_index = (H_D - claculate_H_D_A(data, index)) +alpha*(len(Counter(data.x.T[index]).keys())-1)
        if max_g < g_D_index:
            max_index = index
            max_g = g_D_index
    if max_g!=0:
        print('max_g:', max_g)
    return max_index, max_g


def create_tree(data, index_list, alpha,_e=0.1):
    if len(Counter(data.y)) <= 1 or len(data.y) == 0 or len(index_list) <= 1:
        return node(data, index_list)
    else:
        f_index, f_g = claculate_max_g(data, index_list,alpha)
        if f_g <= _e:
            return node(data, index_list)
        print(f_index, index_list)
        index_list.remove(f_index)
        branch = condition_node(
            f_index=f_index, index_list=index_list, data=data,alpha=alpha)
        branch.create_nodes(data)
        return branch



def predict(root, x):
    result = []
    for xi in x:
        locate = root
        while(1):
            try:
                locate = locate.branches[xi[locate.f_index]]
            except:
                result.append(locate.clas)
                break
    return result



In [12]:
data = s_data(X, y)
index_list = (list(range(data.x.shape[1])))
decision_tree = create_tree(data, index_list,0.2)
pre_y = predict(decision_tree,data.x)
print((np.array(pre_y) == data.y).sum() / len(pre_y))

max_g: 9.402510220562348
2 [0, 1, 2, 3]
max_g: 1.5767701612564369
0 [0, 1, 3]
max_g: 1.4730116670092566
1 [1, 3]
0.9733333333333334
